# Instructions

1. **Set file paths and options** in the **Setup** cell:
   - `iptm_file_path`: Path to the IPTM vs. PEAK file (required).
   - `spoc_file_path`: Path to the SPOC score file (optional).
   - `SPOC_analysis`: Set to `True` if you want to do SPOC-based analysis (requires a valid SPOC file), otherwise `False`.
   - `output_dir`: Where to save charts and selected data (defaults to creating an "analysis" folder next to your IPTM file).

2. **Run the notebook cells in order**:
   - The second cell loads the IPTM data and checks whether to proceed with SPOC or basic analysis.
   - If SPOC analysis is enabled and the file is provided, the subsequent cells will merge data and show the SPOC-based chart.
   - Otherwise, you'll see the basic IPTM vs. PEAK chart.

3. **Interact with the charts**:
   - Use **Lasso/Box select** to label points persistently.
   - Use the **Search** widget to highlight points by partial name.
   - **Clear** labels or search highlights as needed.
   - **Save** the plot as HTML/PDF or **export** selected data as a CSV.

4. **Check the output directory** for your saved files.

In [2]:
# === STEP 1: BASIC SETUP ===

import os
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, Markdown
from plotly.graph_objs import FigureWidget

# ---------------- USER INPUTS ----------------
# Required: path to the IPTM vs. PEAK file
iptm_file_path = "/Volumes/plaschka/shared/alphafold/matthias.vorlaender/screens/transcription_complexes/2025-03-05_PolII_subunits_uniprot_vs_RPB3_FLAG_DNase_vs_wt_DNaseI_without_PDB_IDs/IPTM_vs_PTM.txt"


# Optional: path to the SPOC file
#Set None if not available
spoc_file_path = "/Volumes/plaschka/shared/alphafold/matthias.vorlaender/screens/transcription_complexes/2025-03-05_PolII_subunits_uniprot_vs_RPB3_FLAG_DNase_vs_wt_DNaseI_without_PDB_IDs/spoc_dir_SPOC_analysis.csv"

# Boolean flag indicating whether you want to do SPOC analysis
SPOC_analysis = True  # or False

# Output directory (default is a subfolder 'analysis' next to the IPTM file)
# If you want to override, set output_dir = "/your/desired/output"
default_base = os.path.dirname(iptm_file_path)  # Folder of the IPTM file
default_out = os.path.join(default_base, "analysis")
output_dir = default_out


In [3]:
# If you want to override, set output_dir = "/your/desired/output"
default_base = os.path.dirname(iptm_file_path)  # Folder of the IPTM file
default_out = os.path.join(default_base, "analysis")


# Make sure the output directory exists
os.makedirs(output_dir, exist_ok=True)

print("IPTM file path   :", iptm_file_path)
print("SPOC file path   :", spoc_file_path)
print("SPOC_analysis    :", SPOC_analysis)
print("Output directory :", output_dir)

# === STEP 2: LOAD IPTM & BRANCH ===

# Load the IPTM vs. PEAK data
df_iptm = pd.read_csv(iptm_file_path, sep="\t")
print("Loaded IPTM DataFrame with shape:", df_iptm.shape)

if SPOC_analysis and spoc_file_path is not None:
    print("SPOC analysis is True, and a SPOC file is provided. We will proceed with SPOC-based code.")
else:
    print("Either SPOC_analysis is False or no SPOC file is provided.")
    print("Proceed with Basic Bubble Chart (equivalent to old cell #2).")

# === STEP 3: SPOC MERGE & HOVER SETUP ===
if SPOC_analysis and spoc_file_path is not None:
    print("Loading SPOC file and merging with IPTM data...")
    df_spoc = pd.read_csv(spoc_file_path)
    print("SPOC DataFrame shape:", df_spoc.shape)
    
    # Merge the two DataFrames
    merged_df = pd.merge(
        df_iptm,
        df_spoc,
        left_on="NAME",
        right_on="complex_name",
        how="left"
    )
    print("Merged DataFrame shape:", merged_df.shape)
    
    # === 1) Create an "opacity" column based on spoc_score ===
    if "spoc_score" in merged_df.columns and merged_df["spoc_score"].notnull().any():
        min_score = merged_df["spoc_score"].min()
        max_score = 1.0  # forcing maximum to 1.0
        def compute_opacity(score):
            if pd.isnull(score):
                return 0.1
            if max_score == min_score:
                return 1.0
            return 0.1 + (score - min_score) / (max_score - min_score) * (1.0 - 0.1)
        merged_df["opacity"] = merged_df["spoc_score"].apply(compute_opacity)
    else:
        merged_df["opacity"] = 1.0

    # === 2) Parse short name from "NAME" and store in new column ===
    def parse_shortname(full_name):
        """
        Given something like:
          "76_sp-Q92610-ZN592_HUMAN_vs_sp-Q13889-TF2H3_HUMAN"
        Extract the short name from the target portion
          -> "TF2H3"
        """
        if pd.isnull(full_name):
            return None
        try:
            left_vs_right = full_name.split("_vs_")
            target_part = left_vs_right[1]  # e.g. "sp-Q13889-TF2H3_HUMAN"
            chunks = target_part.split("-")
            if len(chunks) < 3:
                return target_part
            # e.g. chunks[2] = "TF2H3_HUMAN"
            return chunks[2].split("_")[0]  # "TF2H3"
        except:
            return None

    merged_df["protein_name_hit"] = merged_df["NAME"].apply(parse_shortname)

    # === 3) Build default hover text ===
    default_hover_columns = ["NAME", "IPTM", "PEAK", "spoc_score"]
    # We can also add "protein_name_hit" to the default hover if you want
    # default_hover_columns.append("protein_name_hit")

    for col in default_hover_columns:
        if col not in merged_df.columns:
            default_hover_columns.remove(col)

    merged_df["hover_text"] = merged_df.apply(
        lambda row: "<br>".join([f"{col}: {row[col]}" for col in default_hover_columns]),
        axis=1
    )
    
    # === 4) Build the hover selection widget ===
    # This widget uses 'available_hover_columns', which now includes 'protein_name_hit'
    available_hover_columns = list(merged_df.columns)
    # Pre-select defaults
    if default_hover_columns:
        preselected = tuple(default_hover_columns)
    else:
        preselected = (available_hover_columns[0],)  # fallback

    hover_columns_selector = widgets.SelectMultiple(
        options=available_hover_columns,
        value=preselected,
        description="Hover Columns:",
        disabled=False,
        layout={'width': '400px'}
    )

    update_hover_button = widgets.Button(
        description="Update Hover Info",
        button_style="primary"
    )
    
    def update_hover_info(b):
        selected_columns = list(hover_columns_selector.value)
        if not selected_columns:
            print("Please select at least one column for hover info.")
            return
        merged_df["hover_text"] = merged_df.apply(
            lambda row: "<br>".join([str(row[col]) for col in default_hover_columns]),
            axis=1
        )
        print("Hover info updated using columns:", selected_columns)
    
    update_hover_button.on_click(update_hover_info)

    display(Markdown("### SPOC Hover-Column Selection"))
    display(widgets.HBox([hover_columns_selector, update_hover_button]))

else:
    print("Skipping SPOC merge and hover setup because SPOC_analysis=False or no SPOC file provided.")

IPTM file path   : /Volumes/plaschka/shared/alphafold/matthias.vorlaender/screens/transcription_complexes/2025-03-05_PolII_subunits_uniprot_vs_RPB3_FLAG_DNase_vs_wt_DNaseI_without_PDB_IDs/IPTM_vs_PTM.txt
SPOC file path   : /Volumes/plaschka/shared/alphafold/matthias.vorlaender/screens/transcription_complexes/2025-03-05_PolII_subunits_uniprot_vs_RPB3_FLAG_DNase_vs_wt_DNaseI_without_PDB_IDs/spoc_dir_SPOC_analysis.csv
SPOC_analysis    : True
Output directory : /Volumes/plaschka/shared/alphafold/matthias.vorlaender/screens/transcription_complexes/2025-03-05_PolII_subunits_uniprot_vs_RPB3_FLAG_DNase_vs_wt_DNaseI_without_PDB_IDs/analysis
Loaded IPTM DataFrame with shape: (794, 10)
SPOC analysis is True, and a SPOC file is provided. We will proceed with SPOC-based code.
Loading SPOC file and merging with IPTM data...
SPOC DataFrame shape: (233, 30)
Merged DataFrame shape: (794, 40)


### SPOC Hover-Column Selection

In [45]:
# === STEP 4a: SPOC-BASED BUBBLE CHART ===
import re
from plotly.colors import sample_colorscale

# Global variable to store the current hover column selection.
# Initialize with the default hover columns.
current_hover_columns = default_hover_columns

def update_hover_info(b):
    global current_hover_columns
    selected_columns = list(hover_columns_selector.value)
    if not selected_columns:
        print("Please select at least one column for hover info.")
        return
    current_hover_columns = selected_columns
    merged_df["hover_text"] = merged_df.apply(
        lambda row: "<br>".join([str(row[col]) for col in selected_columns]),
        axis=1
    )
    print("Hover info updated using columns:", selected_columns)

update_hover_button.on_click(update_hover_info)

def parse_name_field(name_str):
    """
    Given a string of form:
       "76_sp-Q92610-ZN592_HUMAN_vs_sp-Q13889-TF2H3_HUMAN"
    return a dict with:
       {
         'index': '76',
         'protein1': 'sp-Q92610-ZN592_HUMAN',
         'protein2': 'sp-Q13889-TF2H3_HUMAN'
       }
    If parsing fails, returns something fallback with empty strings.
    """
    try:
        # Split around '_vs_'
        parts = name_str.split("_vs_")
        left_part = parts[0]  # e.g. "76_sp-Q92610-ZN592_HUMAN"
        right_part = parts[1] # e.g. "sp-Q13889-TF2H3_HUMAN"

        # Now split the left_part on the first underscore, to separate index from protein1
        left_sub = left_part.split("_", 1)
        idx = left_sub[0]  # "76"
        prot1 = left_sub[1]  # "sp-Q92610-ZN592_HUMAN"

        return {
            "index": idx,
            "protein1": prot1,
            "protein2": right_part
        }
    except Exception:
        # If something goes wrong, return placeholders
        return {
            "index": "",
            "protein1": "",
            "protein2": ""
        }

def parse_color(color_str):
    """Converts a hex or rgb(a) color string to (r, g, b)."""
    # This helper is used if you need numeric r,g,b from a string.
    # If you only need to pass e.g. "red" or "#ff0000" to Plotly,
    # you can skip converting to (r,g,b). Plotly can handle them directly.
    if color_str.startswith("#"):
        hex_color = color_str.lstrip("#")
        r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        return r, g, b
    elif color_str.startswith("rgb"):
        nums = re.findall(r'\d+', color_str)
        r, g, b = tuple(int(n) for n in nums[:3])
        return r, g, b
    else:
        # Try named CSS color (e.g. "red", "blue") - Plotly accepts those directly
        return color_str

# Before building the figure, let's ensure 'index' is in merged_df
if SPOC_analysis and spoc_file_path is not None:
    if "index" not in merged_df.columns:
        # Parse once for all rows
        parsed_info = merged_df["NAME"].apply(parse_name_field).apply(pd.Series)
        merged_df = pd.concat([merged_df, parsed_info], axis=1)
        # Ensure 'index' column is treated as a string
        merged_df["index"] = merged_df["index"].astype(str)

    # Build the scatter figure
    fig = px.scatter(
        merged_df,
        x="scaled_PEAKavg",
        y="IPTMavg",
        size="scaled_PEAKavg",
        color="scaled_PEAKavg",
        color_continuous_scale="viridis_r",
        title="(SPOC) IPTM vs. Scaled PEAKavg",
        labels={"IPTMavg": "IPTMavg", "scaled_PEAKavg": "Scaled PEAKavg"}
    )

    # Build color array with custom opacity
    merged_df["scaled_PEAKavg"].max() - merged_df["scaled_PEAKavg"].min()
    
    base_colors = px.colors.sequential.Viridis_r
    rgba_colors = []
    for val, opa in zip(norm, merged_df["opacity"]):
        color_str = sample_colorscale(base_colors, val)[0]  # returns a color string
        try:
            # If you want to forcibly parse color, do parse_color(color_str)
            # but if it's a named color, Plotly can handle the string directly
            # and we only need to insert alpha.
            r, g, b = parse_color(color_str)
            if isinstance(r, str):
                # i.e. the color was "red" or something
                rgba_colors.append(r)  # fallback
            else:
                rgba_colors.append(f"rgba({r},{g},{b},{opa})")
        except:
            # If parse fails for some reason, fallback to black with alpha
            rgba_colors.append(f"rgba(0,0,0,{opa})")

    fig.update_traces(
        marker=dict(color=rgba_colors),
        customdata=merged_df[["hover_text"]].values,
        hovertemplate="%{customdata[0]}<extra></extra>"
    )
        
    fig.update_layout(
        # Update x and y axes: no grid, with black axis lines.
        xaxis=dict(
            showgrid=False,
            showline=True,
            linewidth=2,
            linecolor='black'
        ),
        yaxis=dict(
            showgrid=False,
            showline=True,
            linewidth=2,
            linecolor='black'
        ),
        # Add a rectangle shape as an outer border.
        shapes=[
            dict(
                type="rect",
                xref="paper", yref="paper",
                x0=0, y0=0, x1=1, y1=1,
                line=dict(color="black", width=2)
            )
        ],
        # Optionally, set the template and margins.
        template="plotly_white",
        margin=dict(l=50, r=50, t=50, b=50)
    )
        
    figw = FigureWidget(fig)

    # --- GLOBAL STORAGE FOR SELECTIONS ---
    global_persisted_indices_spoc = set()

    def handle_selection(trace, points, selector):
        global global_persisted_indices_spoc
        global_persisted_indices_spoc.update(points.point_inds)
        if not global_persisted_indices_spoc:
            print("No points selected.")
            return
        print("Accumulated selected indices:", global_persisted_indices_spoc)
        selected_df = merged_df.iloc[list(global_persisted_indices_spoc)]
        
        # Example label: extract a short uniprot name from 'NAME' or just show the "index"
        def process_name(name_str):
            try:
                parts = name_str.split("_vs_")
                if len(parts) < 2:
                    return name_str
                # E.g. "sp-Q13889-TF2H3_HUMAN"
                hit = parts[1]
                hit_parts = hit.split("-")
                if len(hit_parts) < 3:
                    return hit
                return hit_parts[2].split("_")[0]
            except:
                return name_str
        
        labels = selected_df["NAME"].apply(process_name)
        
        # Check if we already have a "Persistent Labels" trace
        persistent_trace = None
        for t in figw.data:
            if t.name == "Persistent Labels":
                persistent_trace = t
                break
        
        if persistent_trace is None:
            figw.add_scatter(
                x=selected_df["scaled_PEAKavg"],
                y=selected_df["IPTMavg"],
                mode="text",
                text=labels,
                textposition="top center",
                name="Persistent Labels",
                hoverinfo="skip",
                textfont=dict(color="black", size=12)
            )
        else:
            persistent_trace.x = selected_df["scaled_PEAKavg"]
            persistent_trace.y = selected_df["IPTMavg"]
            persistent_trace.text = labels

    # Attach selection callback
    for trace in figw.data:
        trace.on_selection(handle_selection)

    # --- STANDARD SEARCH (by substring) WIDGETS ---
    search_input_spoc = widgets.Text(
        value="",
        placeholder="Enter partial name to search",
        description="Search NAME:",
        style={'description_width': '120px'},
        layout={'width': '400px'}
    )
    search_button_spoc = widgets.Button(
        description="Search",
        tooltip="Search partial matches",
        button_style="primary"
    )
    clear_search_button_spoc = widgets.Button(
        description="Clear Search",
        tooltip="Remove search highlights",
        button_style="warning"
    )

    def on_search_button_click_spoc(b):
        query = search_input_spoc.value.strip()
        if not query:
            print("Please enter a search query.")
            return
        mask = merged_df["NAME"].str.contains(query, case=False, na=False)
        matched = merged_df[mask]
        if matched.empty:
            print("No matches found.")
            return
        
        # Add highlight scatter
        figw.add_scatter(
            x=matched["scaled_PEAKavg"],
            y=matched["IPTMavg"],
            mode="markers+text",
            marker=dict(symbol="circle-open", size=12, line=dict(width=2, color="red")),
            text=[query]*len(matched),
            textposition="top center",
            name="Search Highlight",
            hoverinfo="skip"
        )
        print(f"Found {len(matched)} match(es). Highlights added.")

    def on_clear_search_button_click_spoc(b):
        indices_to_remove = [i for i, t in enumerate(figw.data) if t.name == "Search Highlight"]
        if not indices_to_remove:
            print("No search highlights to clear.")
            return
        for idx in sorted(indices_to_remove, reverse=True):
            figw.data = figw.data[:idx] + figw.data[idx+1:]
        print("Search highlights cleared.")
    
    search_button_spoc.on_click(on_search_button_click_spoc)
    clear_search_button_spoc.on_click(on_clear_search_button_click_spoc)

    # --- NEW MULTI-GROUP INDEX + COLOR HIGHLIGHT ---
    # Example input: (1,5,12,19=green) (2,9,200=red)
    group_highlight_input_spoc = widgets.Text(
        ##TRICK THE COPY PASTE BIUG HERE!!###
        value="(322,245,22,250,743,690,261,233,229,479,464,107,1,203,660,659,648,363,462,474,475,492,271,192,97=green) (33=grey) (591,425,761,771,286,385,233,479,464,203,660,659,648,462,474,492,192,508,181,190,64,579,40,708,364,416,35,151=red) (436,117,630,573,3,60,687=black)(400,411,423=blue)",
        placeholder="(192,97=green) (35,151=red)",
        description="Multi-Groups:",
        style={'description_width': '100px'},
        layout={'width': '600px'}
    )
    group_highlight_button_spoc = widgets.Button(
        description="Highlight Groups",
        tooltip="Highlight multiple index groups, each with a color",
        button_style="info"
    )

    def on_group_highlight_button_click_spoc(b):
        """
        Example input: (743=green) (385,23,151=red) (20,21,423=blue)
        Each group is parsed, and for each group we use the protein_name_hit value for labeling.
        """
        input_str = group_highlight_input_spoc.value.strip()
        if not input_str:
            print("No group spec given. Format: (1,5,12=red) (2,9=green)")
            return

        # Split by closing parenthesis, filtering out empties.
        group_specs = [chunk.strip() for chunk in input_str.split(")") if chunk.strip()]

        for spec in group_specs:
            # Remove any leading "(" if present.
            if spec.startswith("("):
                spec = spec[1:].strip()

            # Split on "=" to separate indices from the color.
            if "=" in spec:
                left_part, color_part = spec.split("=", 1)
                indices_str = left_part.strip()
                color_str = color_part.strip()
            else:
                indices_str = spec
                color_str = "red"  # default

            # Split indices (comma-separated).
            idx_list = [x.strip() for x in indices_str.split(",") if x.strip()]
            if not idx_list:
                print(f"No valid indices found in '{spec}'")
                continue

            # Find all matching rows in merged_df.
            matched = merged_df[merged_df["index"].isin(idx_list)]
            if matched.empty:
                print(f"No match for indices {idx_list}")
                continue

            # Use the protein_name_hit column for labeling.
            group_label = matched["protein_name_hit"]

            # Add one scatter trace for the group.
            figw.add_scatter(
                x=matched["scaled_PEAKavg"],
                y=matched["IPTMavg"],
                mode="markers+text",
                marker=dict(symbol="circle", color=color_str, size=12),
                text=group_label,
                textposition="top center",
                name=f"Highlight",
                hoverinfo="skip"
            )
            print(f"Highlighted indices {idx_list} in color '{color_str}'")

        print("Group highlight done.")

    group_highlight_button_spoc.on_click(on_group_highlight_button_click_spoc)

    # --- CLEAR LABELS & SAVE PLOT ---
    clear_labels_button_spoc = widgets.Button(
        description="Clear Labels",
        button_style="warning"
    )
    def on_clear_labels_click_spoc(b):
        global_persisted_indices_spoc
        global_persisted_indices_spoc.clear()

        # Remove the "Persistent Labels" or highlight traces if needed
        names_to_remove = ["Persistent Labels"]
        # Also remove any highlight traces we might want to clear
        # If you only want to remove the "Persistent Labels", 
        # leave out highlight traces from the list above.
        indices_to_remove = [
            i for i, t in enumerate(figw.data) 
            if t.name in names_to_remove or t.name.startswith("Highlight ")
        ]
        for idx in sorted(indices_to_remove, reverse=True):
            figw.data = figw.data[:idx] + figw.data[idx+1:]
        print("Persistent labels and highlight traces cleared.")

    clear_labels_button_spoc.on_click(on_clear_labels_click_spoc)

    save_plot_button_spoc = widgets.Button(
        description="Save Plot (HTML & PDF)",
        tooltip="Save the current plot",
        button_style="info"
    )


    file_name_widget_spoc = widgets.Text(
        value="selected_data_spoc.csv",
        placeholder="Enter file name",
        description="Save CSV as:",
        disabled=False
    )
    save_data_button_spoc = widgets.Button(
        description="Save Data",
        button_style="success"
    )
    save_data_output_spoc = widgets.Output()
    
    
    import datetime

    custom_suffix_save = widgets.Text(
        value="",
        placeholder="Add file name suffix",
        description="Filename Suffix:",
        layout={'width': '400px'}
    )
    
    def on_save_plot_click_spoc(b):
        try:
            # Retrieve custom suffix from widget and current timestamp.
            suffix = custom_suffix_save.value.strip()
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            if suffix:
                suffix_str = f"_{suffix}_{timestamp}"
            else:
                suffix_str = f"_{timestamp}"
            
            # Build file paths with the custom suffix and timestamp.
            html_filename = f"spoc_bubble_chart{suffix_str}.html"
            pdf_filename = f"spoc_bubble_chart{suffix_str}.pdf"
            html_path = os.path.join(output_dir, html_filename)
            pdf_path = os.path.join(output_dir, pdf_filename)
            
            # Save the figure.
            figw.write_html(html_path)
            figw.write_image(pdf_path, format="pdf")
            print(f"Plot saved:\n  HTML: {html_path}\n  PDF: {pdf_path}")
        except Exception as e:
            print("Error saving plot:", e)

    save_plot_button_spoc.on_click(on_save_plot_click_spoc)

    # --- DISPLAY ---
    instructions_text_spoc = """
    **SPOC-Based Plot Instructions:**
    1. **Use Lasso/Box select to pick points and persist labels**.
    2. (Optional) Search by partial `NAME` using the first box, then clear highlights if needed.
    3. **Highlight by index** (the digits before the underscore) using the second box, 
       e.g. (1,5,12,19=green) (2,9,200=red) (20=blue)
    4. Clear persistent labels and/or highlight traces if needed.
    5. Save the plot (HTML & PDF) or selected data (CSV).
    """
    display(Markdown(instructions_text_spoc))
    display(widgets.HBox([search_input_spoc, search_button_spoc, clear_search_button_spoc]))
    
    # The new multi-index highlight input
    # Now just display the new widgets:
    display(widgets.HBox([group_highlight_input_spoc, group_highlight_button_spoc]))
    display(widgets.HBox([clear_labels_button_spoc]))

    display(widgets.HBox([custom_suffix_save, save_plot_button_spoc]))


    display(figw)
    display(widgets.HBox([file_name_widget_spoc, save_data_button_spoc]))
    display(save_data_output_spoc)

else:
    print("Skipping SPOC-based bubble chart...")


    **SPOC-Based Plot Instructions:**
    1. **Use Lasso/Box select to pick points and persist labels**.
    2. (Optional) Search by partial `NAME` using the first box, then clear highlights if needed.
    3. **Highlight by index** (the digits before the underscore) using the second box, 
       e.g. (1,5,12,19=green) (2,9,200=red) (20=blue)
    4. Clear persistent labels and/or highlight traces if needed.
    5. Save the plot (HTML & PDF) or selected data (CSV).
    

FigureWidget({
    'data': [{'customdata': array([['NAME: 690_sp-P53803-RPAB4_HUMAN_vs_sp-Q9Y3X0-CCDC9_HUMAN<br>IPTM: 0.688:0.66:0.638:0.615:0.603<br>PEAK: 2.3:3.01:2.98:3.41:4.46<br>spoc_score: 0.255'],
                                   ['NAME: 659_sp-P52435-RPB11_HUMAN_vs_sp-Q9H0E3-SP130_HUMAN<br>IPTM: 0.77:0.732:0.48:0.374:0.201<br>PEAK: 2.79:3.3:9.33:15.08:19.64<br>spoc_score: 0.257'],
                                   ['NAME: 203_sp-O15514-RPB4_HUMAN_vs_sp-Q9H0E3-SP130_HUMAN<br>IPTM: 0.684:0.645:0.581:0.4:0.372<br>PEAK: 2.73:4.48:3.02:8.73:13.18<br>spoc_score: 0.255'],
                                   ...,
                                   ['NAME: 186_sp-O15514-RPB4_HUMAN_vs_sp-O14880-MGST3_HUMAN<br>IPTM: 0.118:0.0794:0.0776:0.0753:0.0665<br>PEAK: 16.31:23.05:20.69:20:21.75<br>spoc_score: nan'],
                                   ['NAME: 357_sp-P62487-RPB7_HUMAN_vs_sp-O14880-MGST3_HUMAN<br>IPTM: 0.0875:0.0875:0.0822:0.073:0.0647<br>PEAK: 21.02:18.05:21.81:24.47:23.66<br>spoc_

Output()

## Note
TO tricky the copy apste button, search the code for "##TRICK THE COPY PASTE BIUG HERE!!###" and enter your hiohglits there!
 
